In [70]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf 
import os
from datetime import datetime
from pathlib import Path

# uplading mnist data 

old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

tf.logging.set_verbosity(old_v)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [71]:
# Setup the Model Parameters
INPUT_SIZE=784
HIDDEN_SIZE=1000
OUTPUT_SIZE = 10  
START_LEARNING_RATE=1e-3
BATCH_SIZE=25
NUM_TRAINING_STEPS = 50
EPOCHS=200
TEST_LENGTH=125
DISPLAY_STEP=25
weight_learning_rate=1e-3

log_dir = os.environ['HOME']+"/MyData/KeRNL/logs/ffn/kernl_mnist_eta_weight_%1.0e_batch_%1.0e_hum_hidd_%1.0e_steps_%1.0e_run_%s" %(weight_learning_rate,BATCH_SIZE,HIDDEN_SIZE,NUM_TRAINING_STEPS, datetime.now().strftime("%Y%m%d_%H%M"))
log_dir

'/home/eghbal/MyData/KeRNL/logs/ffn/kernl_mnist_eta_weight_1e-03_batch_2e+01_hum_hidd_1e+03_steps_5e+01_run_20190304_0953'

In [72]:
def drelu(x):
    return 1 - tf.maximum(0.0, tf.sign(-x))


def dtanh(x):
    return(1-tf.mul(tf.nn.tanh(x),tf.nn.tanh(x)))
  

In [73]:
tf.reset_default_graph()
graph=tf.Graph()
with graph.as_default():

    # define weights and inputs to the network
    X = tf.placeholder('float', shape=[None, INPUT_SIZE])  
    Y = tf.placeholder('float', shape=[None, OUTPUT_SIZE])
    initializer = tf.random_normal_initializer(stddev=0.1)
    # define a function for extraction of variable names
     # Hidden Layer Variables
    W_1 = tf.get_variable("Hidden_W", shape=[INPUT_SIZE, HIDDEN_SIZE], initializer=initializer)
    b_1 = tf.get_variable("Hidden_b", shape=[HIDDEN_SIZE], initializer=initializer)
  # output layer variables 
    W_2 = tf.get_variable("Output_W", shape=[HIDDEN_SIZE, OUTPUT_SIZE], initializer=initializer)
    b_2 = tf.get_variable("Output_b", shape=[OUTPUT_SIZE], initializer=initializer)
    # return weight 
    B=tf.get_variable('B',shape=[OUTPUT_SIZE,HIDDEN_SIZE],initializer=tf.initializers.random_uniform(minval=-0.5,maxval=0.5))
    trainables=[W_1,b_1,W_2,b_2,B]
    variable_names=[v.name for v in tf.trainable_variables()]
    #
    #define transformation from input to output  
  # Hidden Layer Transformation
    g_hidden=tf.matmul(X, W_1) + b_1
    hidden = tf.nn.relu(g_hidden)
  # Output Layer Transformation
    output = tf.matmul(hidden, W_2) + b_2
    

            ##################
            ## kernl train ####
            ##################
    with tf.name_scope("kernl_train") as scope:
        loss = tf.losses.mean_squared_error(Y, output)
        correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(output, 1))
        accuracy = 100 * tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        optimizer  = tf.train.AdamOptimizer(learning_rate=weight_learning_rate)
  # compute and apply gradiants
        dW_2=tf.reduce_mean(tf.transpose(tf.einsum('uv,un->uvn',tf.subtract(output,Y),(hidden))),axis=-1)
        db_2=tf.reduce_mean(tf.subtract(output,Y),axis=0)
        dg_hidden=drelu(g_hidden)
        dg_hidden_diag=tf.linalg.diag(dg_hidden)
        dW_1=tf.transpose(tf.reduce_mean(tf.einsum('uv,ug->uvg',tf.einsum('uv,uvg->ug',tf.einsum('un,nv->uv',tf.subtract(output,Y),B),dg_hidden_diag),X),axis=0))
        db_1=tf.transpose(tf.reduce_mean(tf.einsum('uv,ug->ug',tf.einsum('un,nv->uv',tf.subtract(output,Y),B),dg_hidden),axis=0)) 
        # gradient for B
        dB=tf.negative(tf.reduce_mean(tf.einsum('uv,uz->uvz',output,tf.subtract(hidden,tf.einsum('uv,vz->uz',output,B))),axis=0))
        new_ffn_grads=list(zip([dW_1,db_1,dW_2,db_2,dB],trainables))
        ffn_train_op=optimizer.apply_gradients(new_ffn_grads)
        
  
    with tf.name_scope("evaluate") as scope: 
        kernl_loss_cross_validiation=tf.losses.mean_squared_error(Y,output)
        kernl_correct_pred_cross_val=tf.equal(tf.argmax(Y, 1), tf.argmax(output, 1))
        kernl_accu_cross_validation=100 * tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    with tf.name_scope('cross_validation_summary') as scope: 
        tf.summary.scalar('cross_validation_loss',kernl_loss_cross_validiation+1e-10)
        tf.summary.scalar('cross_validation_accu',kernl_accu_cross_validation+1e-10)
      
        kernl_evaluate_summary_op=tf.summary.merge_all(scope="cross_validation_summary") 
        
                ##################
                # SUMMARIES ######
                ##################
                
    with tf.name_scope("summaries") as scope:
                    # kernl kernel
        tf.summary.histogram('kernl_hidd_W',W_1+1e-10)
        tf.summary.histogram('return_B',B+1e-10)
                    # kernl output weight
        tf.summary.histogram('kernl_output_W',W_2+1e-10)
                    # kernl output bias
                    # kernl loss and accuracy
        tf.summary.scalar('loss_output_prediction',loss+1e-10)
        tf.summary.scalar('accuracy',accuracy+1e-10)
        kernl_merged_summary_op=tf.summary.merge_all(scope="summaries")          
        
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [74]:
# verify initialization
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(graph=graph,) as sess : 
    sess.run(init)
    values,trainable_vars = sess.run([variable_names,trainables])
    for k, v in zip(variable_names,values):
        print(["variable: " , k])
        #print(["value: " , v])
        print(["variable: " , np.unicode_.find(k,'output')]) 
        print(["shape: " , v.shape])
        #print(v) training_steps

['variable: ', 'Hidden_W:0']
['variable: ', -1]
['shape: ', (784, 1000)]
['variable: ', 'Hidden_b:0']
['variable: ', -1]
['shape: ', (1000,)]
['variable: ', 'Output_W:0']
['variable: ', -1]
['shape: ', (1000, 10)]
['variable: ', 'Output_b:0']
['variable: ', -1]
['shape: ', (10,)]
['variable: ', 'B:0']
['variable: ', -1]
['shape: ', (10, 1000)]


In [75]:
Path(log_dir).mkdir(exist_ok=True, parents=True)
filelist = [ f for f in os.listdir(log_dir) if f.endswith(".local") ]
for f in filelist:
    os.remove(os.path.join(log_dir, f))

In [76]:
# write graph into tensorboard 
tb_writer = tf.summary.FileWriter(log_dir,graph)
# run a training session 
with tf.Session(graph=graph) as sess:
    sess.run(init)
    for epoch in range(EPOCHS):
        for step in range(NUM_TRAINING_STEPS): 
            batch_x, batch_y = mnist.train.next_batch(BATCH_SIZE)
            _, kernl_loss,kernl_accu,kernl_merged_summary=sess.run([ffn_train_op,loss,accuracy,kernl_merged_summary_op], feed_dict={X: batch_x, Y: batch_y})
            tb_writer.add_summary(kernl_merged_summary, global_step=epoch*NUM_TRAINING_STEPS+step+1)

            if step % DISPLAY_STEP==0 : 
                print('Epoch: {}, Batch: {},kernl train Loss: {:.3f}, kernl_accuracy : {:.3f}'.format(epoch+1,step + 1, kernl_loss,kernl_accu))
                
        # run test at the end of each epoch 
        test_x=mnist.test.images[:TEST_LENGTH]
        test_y=mnist.test.labels[:TEST_LENGTH]  
        kernl_test_loss,kernl_test_accu, kernl_evaluate_summary=sess.run([kernl_loss_cross_validiation,kernl_accu_cross_validation,kernl_evaluate_summary_op], feed_dict={X: test_x, Y: test_y})        
        tb_writer.add_summary(kernl_evaluate_summary, global_step=epoch*NUM_TRAINING_STEPS+NUM_TRAINING_STEPS+1)
        print('Epoch: {}, cross validation loss :{:.3f}, cross validation accuracy: {:.3f}'.format(epoch+1,kernl_test_loss,kernl_test_accu))
            

    print("Optimization Finished!")
    save_path = saver.save(sess, log_dir+"/model.ckpt", global_step=step,write_meta_graph=True)
    print("Model saved in path: %s" % save_path)


Epoch: 1, Batch: 1,kernl train Loss: 7.653, kernl_accuracy : 16.000
Epoch: 1, Batch: 26,kernl train Loss: 1.792, kernl_accuracy : 16.000
Epoch: 1, cross validation loss :1.307, cross validation accuracy: 15.200
Epoch: 2, Batch: 1,kernl train Loss: 1.429, kernl_accuracy : 8.000
Epoch: 2, Batch: 26,kernl train Loss: 1.022, kernl_accuracy : 16.000
Epoch: 2, cross validation loss :0.859, cross validation accuracy: 29.600
Epoch: 3, Batch: 1,kernl train Loss: 0.735, kernl_accuracy : 24.000
Epoch: 3, Batch: 26,kernl train Loss: 0.844, kernl_accuracy : 16.000
Epoch: 3, cross validation loss :0.661, cross validation accuracy: 30.400
Epoch: 4, Batch: 1,kernl train Loss: 0.589, kernl_accuracy : 28.000
Epoch: 4, Batch: 26,kernl train Loss: 0.467, kernl_accuracy : 40.000
Epoch: 4, cross validation loss :0.557, cross validation accuracy: 25.600
Epoch: 5, Batch: 1,kernl train Loss: 0.622, kernl_accuracy : 32.000
Epoch: 5, Batch: 26,kernl train Loss: 0.450, kernl_accuracy : 28.000
Epoch: 5, cross vali